In [1]:
import sqlite3

### Connect to the database

In [6]:
conn = sqlite3.connect('../max-experiments/itinerary-scraping/journeys.db')
cursor = conn.cursor()

### Outage durations

In [24]:
cursor.execute('SELECT * FROM outages')
outages = cursor.fetchall()

cursor.execute('SELECT * FROM journeys')
journeys = cursor.fetchall()

# number of outages
print(len(outages))

# frequency of outages
from collections import Counter
outage_freq = Counter([outage[1] for outage in outages])
print(outage_freq)

print('---')
print("% of journeys with outages:", round(len(outages)/len(journeys) * 100, 3))

183
Counter({'stop_point:tcl:SP:46024': 8, 'stop_point:tcl:SP:30205': 5, 'stop_point:tcl:SP:46051': 5, 'stop_point:tcl:SP:46033': 5, 'stop_point:tcl:SP:46036': 4, 'stop_point:tcl:SP:48137': 4, 'stop_point:tcl:SP:30209': 4, 'stop_point:tcl:SP:30189': 4, 'stop_point:tcl:SP:30197': 4, 'stop_point:tcl:SP:46058': 4, 'stop_point:tcl:SP:30445': 4, 'stop_point:tcl:SP:30211': 4, 'stop_point:tcl:SP:30156': 3, 'stop_point:tcl:SP:46053': 3, 'stop_point:tcl:SP:46054': 3, 'stop_point:tcl:SP:46026': 3, 'stop_point:tcl:SP:46028': 3, 'stop_point:tcl:SP:30448': 3, 'stop_point:tcl:SP:30198': 3, 'stop_point:tcl:SP:30200': 3, 'stop_point:tcl:SP:30208': 3, 'stop_point:tcl:SP:46060': 3, 'stop_point:tcl:SP:46025': 3, 'stop_point:tcl:SP:42744': 3, 'stop_point:tcl:SP:42738': 3, 'stop_point:tcl:SP:30185': 3, 'stop_point:tcl:SP:36394': 3, 'stop_point:tcl:SP:30191': 3, 'stop_point:tcl:SP:30199': 3, 'stop_point:tcl:SP:30103': 3, 'stop_point:tcl:SP:30447': 3, 'stop_point:tcl:SP:30210': 3, 'stop_point:tcl:SP:46031': 